In [ ]:
zone_colors = {
    "residential": "#FFD700",  # жёлтый
    "industrial": "#6A5ACD",  # синий в фиолетовый
    "business": "#FF8C00",  # оранжевый
    "recreation": "#ADFF2F",  # светло-зелёный
    "transport": "#A9A9A9",  # серый
    "agriculture": "#20B2AA",  # сине-зелёный
    "special": "#8B4513",  # коричневый
}
zone_translation = {
    "residential": "Жилые зоны",
    "industrial": "Промышленные зоны",
    "business": "Общественно-деловые зоны",
    "recreation": "Рекреационные зоны",
    "transport": "Зоны транспортной инфраструктуры",
    "agriculture": "Зоны сельскохозяйственного назначения",
    "special": "Зоны специального назначения",
}
wrapped_translation = {
    "residential": "Жилые зоны",
    "business": "Общественно-\nделовые зоны",
    "recreation": "Рекреационные зоны",
    "transport": "Зоны транспортной\nинфраструктуры",
    "agriculture": "Зоны\nсельскохозяйственного\nназначения",
    "special": "Зоны специального\nназначения",
    "industrial": "Промышленные зоны",
}

In [ ]:
%load_ext autoreload
%autoreload 2

from app.gen_planner.python.src.utils import territory_splitter
from app.gen_planner.python.src import GenPlanner, basic_func_zone, func_zones, terr_zones
import geopandas as gpd
import os
from shapely import Point
import pandas as pd

if os.path.exists("rust_data/conv.csv"):
    os.remove("rust_data/conv.csv")
territory = gpd.read_file('../scenarios_data/project Шлиссельбург.geojson')

middle_lower = Point(31.018, 59.917)
right_upper = Point(31.046, 59.922)
right_lower = Point(31.043, 59.911)
left_upper = Point(30.999, 59.927)
left_lower = Point(30.993, 59.912)

zones_ration = {
    "residential": 0.5714285714285714,
    "business": 0.1142857142857143,
    "recreation": 0.1142857142857143,
    "transport": 0.08571428571428573,
    "special": 0.05714285714285715,
    "agriculture": 0.05714285714285715,
}

fixed_zones = gpd.GeoDataFrame(
    {'zone': [
        "business",
        "recreation",
        "agriculture",
        "special"]},
    geometry=[right_upper, right_lower, left_upper, left_lower],
    crs=4326)

generated_zones, generated_roads = GenPlanner(territory, dev_mod=True).split_features(zones_ratio_dict=zones_ration,
                                                                                      fixed_zones=fixed_zones)

m1 = pd.concat(
    [generated_zones, fixed_zones.rename(columns={'zone': 'zone_name'}).to_crs(generated_zones.crs)]).explore(
    tiles='Cartodb Positron',
    column='zone_name'
    , cmap='Dark2')
# generated_roads.explore(m=m1,column='road_lvl')
m1

In [ ]:
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import contextily as ctx
from matplotlib.lines import Line2D

# Перевод в Web Mercator (обязательно для contextily)
territory = territory.to_crs(epsg=3857)
fixed_zones = fixed_zones.to_crs(epsg=3857)
# Границы
fig, ax = plt.subplots(figsize=(10, 10))
minx, miny, maxx, maxy = territory.total_bounds
dx, dy = (maxx - minx) * 0.05, (maxy - miny) * 0.05
ax.set_xlim(minx - dx, maxx + dx)
ax.set_ylim(miny - dy, maxy + dy)

# Отображаем базу
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, zoom=15)

# Рисуем территорию
territory.plot(ax=ax, facecolor='#61b0e8', edgecolor='gray', alpha=0.4, linewidth=1.0)
legend_elements = []
for zone, color in zone_colors.items():
    p_gdf = fixed_zones[fixed_zones["zone"] == zone]
    if not p_gdf.empty:
        p_gdf.plot(
            ax=ax,
            facecolor=color,
            markersize=250,
            edgecolor='black',
            linewidth=0.6,
        )
        legend_elements.append(Line2D([0], [0], marker='o', color=color, label=f'Точка фиксации зоны \"{zone_translation.get(zone)}\"', markersize=8))

# Легенда и оформление
ax.legend(handles=legend_elements, loc="upper right")
ax.set_axis_off()
ax.set_title("Исходный полигон", fontsize=16)

plt.tight_layout()
plt.savefig("./svetogorsk/shlis_init.png", dpi=200, bbox_inches="tight")
plt.show()


In [ ]:
from matplotlib.lines import Line2D

from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import contextily as ctx

# Подготовка
generated_zones['name'] = generated_zones['zone_name']

# Перевод в Web Mercator (обязательно для contextily)
territory = territory.to_crs(epsg=3857)
generated_zones = generated_zones.to_crs(epsg=3857)
fixed_zones = fixed_zones.to_crs(epsg=3857)

# Границы
fig, ax = plt.subplots(figsize=(10, 10))
minx, miny, maxx, maxy = territory.total_bounds
dx, dy = (maxx - minx) * 0.05, (maxy - miny) * 0.05
ax.set_xlim(minx - dx, maxx + dx)
ax.set_ylim(miny - dy, maxy + dy)

# Отображаем базу
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, zoom=15)

# Рисуем территорию
territory.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=1.0)

# Зоны + легенда
legend_elements = []
for zone, color in zone_colors.items():
    zone_gdf = generated_zones[generated_zones["name"] == zone]
    if not zone_gdf.empty:
        zone_gdf.plot(
            ax=ax,
            facecolor=color,
            edgecolor='gray',
            linewidth=0.4,
            alpha=0.6,
        )
        legend_elements.append(Patch(facecolor=color, alpha=0.6, edgecolor="gray", label=zone_translation.get(zone)))
    p_gdf = fixed_zones[fixed_zones["zone"] == zone]
    if not p_gdf.empty:
        p_gdf.plot(
            ax=ax,
            facecolor=color,
            markersize=80,
            edgecolor='black',
            linewidth=0.8,
        )
        legend_elements.append(Line2D([0], [0], marker='o', color=color, label='Точка фиксации', markersize=8))

# Легенда и оформление
ax.legend(handles=legend_elements, loc="upper right")
ax.set_axis_off()
ax.set_title("Территориальное зонирование", fontsize=16)

plt.tight_layout()
plt.savefig("./svetogorsk/shlis_zonning.png", dpi=200, bbox_inches="tight")
plt.show()


In [ ]:
from PIL import Image, ImageDraw, ImageFont
import os
import numpy as np


def gif_to_collage(gif_path, num_frames=9, output_path="collage.png", grid=None,
                   collage_size=(2000, 1300), font_path="Raleway-Medium.ttf"):
    gif = Image.open(gif_path)
    total_frames = gif.n_frames

    # Экспоненциальное распределение индексов кадров
    exp_range = np.logspace(0, 1, num=num_frames, base=40.0) - 1
    exp_range = exp_range / exp_range.max() * (total_frames - 1)
    selected_indices = sorted(set(int(i) for i in exp_range))

    # Гарантировать последний кадр
    if (total_frames - 1) not in selected_indices:
        selected_indices[-1] = total_frames - 1

    if grid is None:
        cols = int(num_frames ** 0.5)
        rows = (num_frames + cols - 1) // cols
    else:
        cols, rows = grid

    collage_width, collage_height = collage_size
    cell_width = collage_width // cols
    cell_height = collage_height // rows

    font_size = min(cell_width, cell_height) // 8
    font = ImageFont.truetype(font_path, font_size)

    collage = Image.new("RGB", (collage_width, collage_height), "white")
    draw_collage = ImageDraw.Draw(collage)

    for index, frame_idx in enumerate(selected_indices):
        gif.seek(frame_idx)
        frame = gif.copy().convert("RGB")

        # Сохраняем пропорции при масштабировании
        frame_ratio = frame.width / frame.height
        cell_ratio = cell_width / cell_height

        if frame_ratio > cell_ratio:
            new_width = cell_width
            new_height = int(cell_width / frame_ratio)
        else:
            new_height = cell_height
            new_width = int(cell_height * frame_ratio)

        frame_resized = frame.resize((new_width, new_height), Image.LANCZOS)

        # Центрируем изображение в ячейке
        x_cell = (index % cols) * cell_width
        y_cell = (index // cols) * cell_height
        x_offset = x_cell + (cell_width - new_width) // 2
        y_offset = y_cell + (cell_height - new_height) // 2

        # Вставляем кадр
        collage.paste(frame_resized, (x_offset, y_offset))

        # Добавляем текст
        draw_frame = ImageDraw.Draw(collage)
        draw_frame.text((x_offset + 50, y_offset + 10), f"Итерация {frame_idx}", fill="black", font=font)

        # Рамка по границе ячейки
        draw_collage.rectangle(
            [x_cell, y_cell, x_cell + cell_width - 1, y_cell + cell_height - 1],
            outline="black",
            width=1  # тонкая рамка
        )

    collage.save(output_path)
    print(f"Коллаж сохранён в {output_path}")


# Пример использования
gif_to_collage("test.gif", num_frames=9, output_path="collage.png")


In [ ]:
from app.gen_planner.python.src import terr_zones, TerritoryZone

custom_terr_zone = TerritoryZone(
    "custom",
    10_000,
)
territory = gpd.read_file('../scenarios_data/project ШлиссельбургMulti.geojson')
roads = gpd.read_file('../scenarios_data/roads_shlis.geojson')
roads.geometry = roads.to_crs(32636).buffer(2.5).to_crs(4326)
splitted_territory = territory_splitter(territory, roads)
splitted_territory = splitted_territory[~splitted_territory['is_splitter']]
generated_zones, generated_roads = GenPlanner(splitted_territory).features2blocks(custom_terr_zone)

In [ ]:
m1 = generated_zones.explore(column='territory_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
generated_roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from shapely import Point
from app.gen_planner.python.src import GenPlanner
import geopandas as gpd
import os

if os.path.exists("./rust_data/conv.csv"):
    print('removed conv')
    os.remove("./rust_data/conv.csv")

territory = gpd.read_file('../scenarios_data/project Шлиссельбург.geojson')
zones_ratio_dict = {'right_lower': 0.225, 'right_upper': 0.225, 'left_upper': 0.225, 'left_lower': 0.225,
                    'middle_lower': 0.1, 'middle_upper': 0.1, 'free_zone1': 0.1, 'free_zone2': 0.1}
fixed_zones = gpd.GeoDataFrame()
middle_lower = Point(31.015, 59.911)
middle_upper = Point(31.022, 59.922)
right_upper = Point(31.046, 59.922)
right_lower = Point(31.043, 59.911)
left_upper = Point(30.999, 59.927)
left_lower = Point(30.993, 59.912)
fixed_2zones = gpd.GeoDataFrame(
    {'zone': ['middle_lower', 'middle_upper']},
    geometry=[middle_lower, middle_upper],
    crs=4326)

fixed_1zone2p = gpd.GeoDataFrame(
    {'zone': ['middle_lower', 'middle_lower']},
    geometry=[middle_lower, middle_upper],
    crs=4326)

fixed_6zones = gpd.GeoDataFrame(
    {'zone': ['middle_lower', 'middle_upper', 'right_upper', 'right_lower', 'left_upper', 'left_lower']},
    geometry=[middle_lower, middle_upper, right_upper, right_lower, left_upper, left_lower],
    crs=4326)

fixed_zones = fixed_6zones

generated_zones, generated_roads = GenPlanner(territory, dev_mod=True).split_features(zones_ratio_dict=zones_ratio_dict,
                                                                                      fixed_zones=fixed_zones)

In [ ]:
import pandas as pd

m1 = pd.concat(
    [generated_zones, fixed_zones.rename(columns={'zone': 'zone_name'}).to_crs(generated_zones.crs)]).explore(
    tiles='Cartodb Positron',
    column='zone_name')
# generated_roads.explore(m=m1,column='road_lvl')
m1

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")  # Можешь попробовать "dark", "white", "ticks" и т.д.
# Загрузка данных
df = pd.read_csv("rust_data/conv.csv", header=None, names=[
    "iter", "each_area", "full_area", "wall", "topo", "fix", "lloyd", "group_fix", "lr"
])
df["total"] = df[["each_area", "full_area", "wall", "lloyd", "topo", "group_fix", "fix"]].sum(axis=1)

# Отрисовка графика
plt.figure(figsize=(10, 7))

# Основной total loss
plt.plot(df["iter"], df["total"], label="Сумма потерь", linewidth=2, color="black")

# Остальные компоненты
plt.plot(df["iter"], df["each_area"], label="Площадь каждой зоны", linestyle="--")
plt.plot(df["iter"], df["full_area"], label="Полная площадь", linestyle="--")
plt.plot(df["iter"], df["wall"], label="Ровность стен", linestyle="--")
plt.plot(df["iter"], df["topo"], label="Топология разбиения", linestyle="--")
plt.plot(df["iter"], df["fix"], label="Фиксация", linestyle="--")
plt.plot(df["iter"], df["group_fix"], label="Групповая фиксация", linestyle="--")
plt.plot(df["iter"], df["lloyd"], label="Расслабленность", linestyle="--")

# Оформление
plt.xlabel("Итерация")
plt.ylabel("Потери")
plt.title("Сходимость функций потерь")
plt.legend()
plt.grid(True)
plt.ylim(0, 800)
plt.xlim(left=0)
plt.tight_layout()
plt.savefig("./svetogorsk/Convergence.png", dpi=200, bbox_inches="tight")
plt.show()


In [ ]:
df

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(df["iter"], df["lr"], label="Learning Rate", color="blue", linewidth=2)
plt.xlabel("Iteration")
plt.ylabel("Learning Rate")
plt.title("Learning Rate Schedule (Cosine Decay)")
plt.grid(True, linestyle="--", alpha=0.7)
plt.xlim(left=0)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
roads.geometry = roads.to_crs(32636).buffer(2).to_crs(4326)
splitted_zones = territory_splitter(generated_zones, roads)
splitted_zones[~splitted_zones['is_splitter']].explore(tiles='Cartodb Positron')

In [ ]:
splitted_territory.explore(tiles='Cartodb Positron')

In [ ]:
from app.gen_planner.python.src import GenPlanner, basic_func_zone, func_zones
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
%load_ext autoreload
%autoreload 2

from app.gen_planner.python.src.utils import territory_splitter
from app.gen_planner.python.src import GenPlanner, basic_func_zone, func_zones
import geopandas as gpd

territory = gpd.read_file('project Шлиссельбург.geojson')
roads = gpd.read_file('roads_shlis.geojson')
roads.geometry = roads.to_crs(32636).buffer(5).to_crs(4326)
splitted_territory = territory_splitter(territory, roads)
splitted_territory = splitted_territory[~splitted_territory['is_splitter']].reset_index(drop=True)
generated_zones, generated_roads = GenPlanner(splitted_territory, dev_mod=True).features2terr_zones2blocks(
    func_zones.residential_func_zone)

m1 = generated_zones.explore(column='territory_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
generated_roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
generated_zones[generated_zones['proxy'].isna()].explore(column='territory_zone', tiles='CartoDB positron',
                                                         cmap='Dark2', categorical=True)


In [ ]:
generated_zones[generated_zones['proxy'] == 1].explore(column='zone_name')

In [ ]:
splitted_territory.explore()

In [ ]:
territory = gpd.read_file('project Шлиссельбург.geojson')
# roads = gpd.read_file('roads_shlis.geojson')
# roads.geometry = roads.to_crs(32636).buffer(5).to_crs(4326)
# splitted_territory = territory_splitter(territory, roads)
# splitted_territory = splitted_territory[~splitted_territory['is_splitter']].reset_index(drop=True)
generated_zones, generated_roads = GenPlanner(territory, dev_mod=True).features2terr_zones2blocks(
    func_zones.residential_func_zone)

m1 = generated_zones.explore(column='territory_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
generated_roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)